### Imports

In [56]:
import os
from monai.transforms import (
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    ToTensord,
    SaveImaged,
    Spacingd,
    EnsureTyped,
    AsChannelLastd,
    AsChannelFirstd,
    AsDiscreted,
    EnsureChannelFirstd,
    ScaleIntensityRanged,
    FillHolesd,
    RandCropByLabelClassesd,
    Resized, RandFlipd, RandRotate90d,
    CropForeground,
)
from monai.transforms.transform import MapTransform
from monai.transforms.inverse import InvertibleTransform
from monai.config import DtypeLike, KeysCollection
from monai.config.type_definitions import NdarrayOrTensor
from typing import Any, Dict, Hashable, List, Mapping, Optional, Sequence, Tuple, Union
import numpy as np
from monai.transforms.intensity.array import (
    ScaleIntensityRangePercentiles,
)
import matplotlib.pyplot as plt
from ipywidgets.widgets import * 
import ipywidgets as widgets
import matplotlib.pyplot as plt
import glob 
import torch
import os
# import cv2

### Transformations

In [3]:
class RemoveDicts(MapTransform, InvertibleTransform):

    def __init__(
        self,
        keys: KeysCollection,
        allow_missing_keys: bool = False,
    ) -> None:
        super().__init__(keys, allow_missing_keys)

    def __call__(self, data: Mapping[Hashable, NdarrayOrTensor]) -> Dict[Hashable, NdarrayOrTensor]:
        d = dict(data)
        for key in self.key_iterator(d):
            self.push_transform(d, key)
        # print(d["image_meta_dict"]["filename_or_obj"])
        a = {"image": d["image"], "label": d["label"], "path": d["image_meta_dict"]["filename_or_obj"]}
        # print(a["path"])
        d = a
        return d

    def inverse(self, data: Mapping[Hashable, Any]) -> Dict[Hashable, Any]:
        d = deepcopy(dict(data))
        for key in self.key_iterator(d):
            d[key] = d[key]
            # Remove the applied transform
            self.pop_transform(d, key)
        return d

In [4]:
class NNUnetScaleIntensity(MapTransform):
    """
    Dictionary-based wrapper of :py:class:`monai.transforms.ScaleIntensityRange`.

    Args:
        keys: keys of the corresponding items to be transformed.
            See also: monai.transforms.MapTransform
        a_min: intensity original range min.
        a_max: intensity original range max.
        b_min: intensity target range min.
        b_max: intensity target range max.
        clip: whether to perform clip after scaling.
        dtype: output data type, if None, same as input image. defaults to float32.
        allow_missing_keys: don't raise exception if key is missing.
    """
    def _compute_stats(self, volume, mask):
        volume = volume.copy()
        mask = np.greater(mask, 0) # get only non-zero positive pixels/labels
        volume = volume * mask
        volume = np.ma.masked_equal(volume,0).compressed()
        if len(volume) == 0:
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
        median = np.median(volume)
        mean = np.mean(volume)
        std = np.std(volume)
        mn = np.min(volume)
        mx = np.max(volume)
        percentile_99_5 = np.percentile(volume, 99.5)
        percentile_00_5 = np.percentile(volume, 00.5)
        print(median, mean, std, mn, mx, percentile_99_5, percentile_00_5)
        return median, mean, std, mn, mx, percentile_99_5, percentile_00_5

    def __init__(
        self,
        keys: KeysCollection,
        dtype: DtypeLike = np.float32,
        allow_missing_keys: bool = False,
    ) -> None:
        super().__init__(keys, allow_missing_keys)

    def __call__(self, data: Mapping[Hashable, NdarrayOrTensor]) -> Dict[Hashable, NdarrayOrTensor]:
        d = dict(data)
        for key in self.key_iterator(d):
            stats = self._compute_stats(d[key], d['label'])
            d[key] = np.clip(d[key], stats[6], stats[5])
            d[key] = (d[key] - stats[1]) / stats[2]
        return d

In [5]:
class CustomCropForegroundd(MapTransform):
    def __init__(
        self,
        keys: KeysCollection,
        source_key: str,
        allow_missing_keys: bool = False,
        **np_kwargs,
    ) -> None:
        super().__init__(keys, allow_missing_keys)
        self.source_key = source_key

    def __call__(self, data: Mapping[Hashable, NdarrayOrTensor]) -> Dict[Hashable, NdarrayOrTensor]:
        d = dict(data)
        original_label = d['label']
        unique_labels = np.unique(original_label) 
        # Check if there is one VI or two
        if len(unique_labels > 4) == 1:
            d = CropForegroundd(keys=self.keys, source_key=self.source_key)(d) # crop ROI
        else:
            spleen_labels = np.where((original_label%2 != 0), 0, original_label)
            vi_spleen_labels = np.where((original_label != 6), 0, original_label)
            liver_labels = np.where((original_label%2 == 0), 0, original_label)
            vi_liver_labels = np.where((original_label != 5), 0, original_label)
            new_d = {'image': d['image'], 'label_spleen': spleen_labels, 'label_liver': liver_labels, 'label_vi_spleen': vi_spleen_labels, 'label_vi_liver': vi_liver_labels}
            spleen_d = CropForegroundd(keys=['image', 'label_spleen'], source_key=['label_spleen'])(new_d).pop('label_liver') # crop ROI
            start, end = spleen_d["boundg_box"]
            box_start_spleen, box_end_spleen = CropForeground().compute_bounding_box(img=new_d['label_vi_spleen'])
            box_start_liver, box_end_liver = CropForeground().compute_bounding_box(img=new_d['label_vi_liver'])
            # spleen_d["label"][s:e, :,:]
            # debug to check if the bounding box is correct dimension wise, should have 3 dimensions
            

            liver_d = CropForegroundd(keys=['image', 'label_liver'], source_key=['label_liver'])(new_d).pop('label_spleen') # crop ROI

        return d

In [54]:
from PIL import Image
class WriteToPNG(MapTransform):

    def __init__(
        self,
        keys: KeysCollection,
        output_dir: str,
        mode:str,
        allow_missing_keys: bool = False,
    ) -> None:
        super().__init__(keys, allow_missing_keys)
        self.output_dir = output_dir
        self.mode = mode

    def __call__(self, data: Mapping[Hashable, NdarrayOrTensor]) -> Dict[Hashable, NdarrayOrTensor]:
        d = dict(data)
        for key in self.key_iterator(d):
            if isinstance(d[key], torch.Tensor):
                d[key] = d[key].detach().cpu().numpy()
            for slice in range(d[key].shape[-1]):
                # print("type: " + d[key].dtype)
                filename = os.path.basename(d["image_meta_dict"]["filename_or_obj"]).split(".")[0] + f"_{slice}.png"
                if key == "image":
                    if self.mode == "train":
                        save_dir = os.path.join(self.output_dir, 'imagesTr', filename)
                    else:
                        save_dir = os.path.join(self.output_dir, 'imagesTs', filename)
                else:
                    if self.mode == "train":
                        save_dir = os.path.join(self.output_dir, 'labelsTr', filename)
                    else:
                        save_dir = os.path.join(self.output_dir, 'labelsTs', filename)
                if not os.path.exists(os.path.dirname(save_dir)):
                    print(f"Creating directory: {os.path.dirname(save_dir)}")
                    os.makedirs(os.path.dirname(save_dir))
                print(f"Saving to {save_dir}")
                plt.imsave(save_dir, d[key][0, :, :, slice], cmap="gray")
                # img = Image.fromarray(d[key][0, :, :, slice].astype(np.float32))
                # img.save(save_dir)

        return d

### Dataset

In [55]:
train_images = sorted( glob.glob( os.path.join( "/mnt/chansey/lauraalvarez/","data", "vascular_injuries", "nii", "imagesTr", "*.nii.gz") ) )
train_labels = sorted( glob.glob( os.path.join( "/mnt/chansey/lauraalvarez/","data", "vascular_injuries", "nii", "labelsTr", "*.nii.gz") ) )
data_dicts = [ {"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels) ]
test_images = sorted( glob.glob( os.path.join( "/mnt/chansey/lauraalvarez/","data", "vascular_injuries", "nii", "imagesTs", "*.nii.gz" ) ) )
test_labels = sorted( glob.glob( os.path.join( "/mnt/chansey/lauraalvarez/","data", "vascular_injuries", "nii", "labelsTs", "*.nii.gz") ) )
data_dicts_test = [{"image": image_name, "label": label_name} for image_name, label_name in zip(test_images, test_labels) ]

In [56]:
# transforms_bsl = Compose([ LoadImaged(keys=["image", "label"]), AddChanneld(keys=["image"]), ToTensord(keys=["image", "label"]),])
transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        # RemoveDicts(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        # AsChannelFirstd(keys=["label"]),
        # AsDiscreted(keys=["label"], argmax=True),
        # Orientationd(keys=["image", "label"], axcodes="RAS"),
        # Spacingd( keys=["image", "label"], pixdim=(1.5, 1.5, 1), mode=("bilinear", "nearest"),),
        # CropForegroundd(keys=["image", "label"], source_key="label"),
        CustomCropForegroundd(keys=["image", "label"], source_key="label"),
        NNUnetScaleIntensity(keys=["image"]),
        # ClosePreprocessing(keys=["label"]),
        WriteToPNG(keys=["image", "label"], output_dir="/mnt/chansey/lauraalvarez/data/vascular_injuries/png/", mode="test"),
        ToTensord(keys=["image", "label"]),
    ]
)

# injure_org = transforms_bsl(data_dicts)
error_cases = list()
for data_dict in data_dicts_test:
    try:
        data_dict = transforms(data_dict)
    except Exception as e:
            error_cases.append(data_dict)
print(f"{len(error_cases)} error cases")
print(error_cases)
# injure_crop = transforms(data_dicts)
# print(injure_crop["image"].shape, injure_crop["label"].shape)

0 error cases
[]


In [57]:
blended_true_label = blend_images(injure_crop["image"], injure_crop["label"], alpha=0.9)
blended_final_true_label_closed = blended_true_label.permute(1,2,0,3)
print(blended_final_true_label_closed.shape)

NameError: name 'injure_crop' is not defined

In [ ]:
from monai.visualize import matshow3d, blend_images
import torch 

def dicom_animation(slice):
    plt.figure(figsize=(18, 6))
    plt.title(f"liver no injured ")
    plt.imshow(blended_final_true_label_closed[:, :, :, slice], cmap="bone")
    plt.show()

interact(dicom_animation, slice=(0, blended_final_true_label_closed.shape[-1]-1))

interactive(children=(IntSlider(value=88, description='slice', max=177), Output()), _dom_classes=('widget-inte…

<function __main__.dicom_animation(slice)>

In [ ]:
from monai.visualize import matshow3d, blend_images
import torch 

def dicom_animation(slice):
    plt.figure(figsize=(18, 6))
    plt.title(f"liver no injured ")
    plt.imshow(blended_final_true_label[:, :, :, slice], cmap="bone")
    plt.show()

interact(dicom_animation, slice=(0, blended_final_true_label.shape[-1]-1))

interactive(children=(IntSlider(value=88, description='slice', max=177), Output()), _dom_classes=('widget-inte…

<function __main__.dicom_animation(slice)>